# Explorando "European Soccer Database"
Carlos Natale

# Synopsis
Vamos buscar a associação entre os clubes de futebol da Europa e responder a pergunta: Quais clubes mais realizam transações de compra e venda de jogadores, entre si?
  
# Dataset 
Dataset oferecido pelo Kaggle: 
https://www.kaggle.com/hugomathien/soccer
  
O dataset contém cerca de 25.000 partidas de onze ligas de futebol europeias a partir da temporada 2008/2009 até a temporada 2015/2016. Depois de realizar o trabalho de Data Wrangling, será gerado um conjunto de dados transacionais adequado para análise de cesta de compras. No total, o conjunto de dados transacionais de futebol contém cerca de 18.000 registros. 

In [1]:
setwd("C:\\Users\\chnat\\anaconda3\\portfolio")
getwd()
options(warn=-1)


[1] "C:/Users/chnat/anaconda3/portfolio"

In [ ]:
# Pacotes
install.packages("RSQLite")
install.packages("dplyr")
install.packages("tidyr")
install.packages("arules")
install.packages("arulesSequences")
install.packages("readr")
install.packages("visNetwork")
install.packages("igraph")
install.packages("lubridate")
install.packages("DT")

In [2]:
library(RSQLite)
library(dplyr)
library(tidyr)
library(arules)
library(arulesSequences)
library(readr)
library(stringr)
library(visNetwork)
library(igraph)
library(lubridate)
library(DT)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: Matrix

Attaching package: 'Matrix'

The following objects are masked from 'package:tidyr':

    expand, pack, unpack


Attaching package: 'arules'

The following object is masked from 'package:dplyr':

    recode

The following objects are masked from 'package:base':

    abbreviate, write


Attaching package: 'igraph'

The following object is masked from 'package:arulesSequences':

    similarity

The following object is masked from 'package:arules':

    union

The following object is masked from 'package:tidyr':

    crossing

The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union


Att

# Os dados estão disponibilizados em um banco de dados SQLITE que pode ser baixado do kaggle pelo link informado acima.

In [3]:
# Conectando no banco de dados
con = dbConnect(RSQLite::SQLite(), dbname="database.sqlite")

In [4]:
# Obtendo a lista de tabelas
alltables = dbListTables(con)
alltables

[1] "Country"           "League"            "Match"            
[4] "Player"            "Player_Attributes" "Team"             
[7] "Team_Attributes"   "sqlite_sequence"

In [5]:
# Extraindo as tabelas
players       = dbReadTable(con, "Player")
players_stats = dbReadTable(con, "Player_Attributes")
teams         = dbReadTable(con, "Team")
league        = dbReadTable(con, "League")
Matches       = dbReadTable(con, "Match")

In [6]:
# Visualizando as tabelas
head(players)
head(players_stats)
head(teams)
head(league)
head(Matches)

id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
6,27316,Aaron Hunt,158138,1986-09-04 00:00:00,182.88,161


id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
1,218353,505942,2016-02-18 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
2,218353,505942,2015-11-19 00:00:00,67,71,right,medium,medium,49,...,54,48,65,69,69,6,11,10,8,8
3,218353,505942,2015-09-21 00:00:00,62,66,right,medium,medium,49,...,54,48,65,66,69,6,11,10,8,8
4,218353,505942,2015-03-20 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
5,218353,505942,2007-02-22 00:00:00,61,65,right,medium,medium,48,...,53,47,62,63,66,5,10,9,7,7
6,189615,155782,2016-04-21 00:00:00,74,76,left,high,medium,80,...,66,59,76,75,78,14,7,9,9,12


id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
1,9987,673,KRC Genk,GEN
2,9993,675,Beerschot AC,BAC
3,10000,15005,SV Zulte-Waregem,ZUL
4,9994,2007,Sporting Lokeren,LOK
5,9984,1750,KSV Cercle Brugge,CEB
6,8635,229,RSC Anderlecht,AND


id,country_id,name
1,1,Belgium Jupiler League
1729,1729,England Premier League
4769,4769,France Ligue 1
7809,7809,Germany 1. Bundesliga
10257,10257,Italy Serie A
13274,13274,Netherlands Eredivisie


id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
6,1,1,2008/2009,1,2008-09-24 00:00:00,492478,8203,8342,1,...,1.67,4.35,3.40,1.70,4.50,3.40,1.70,NA,NA,NA


In [7]:
# Substituindo espaço por underline nos nome muito longos
teams$team_long_name = str_replace_all(teams$team_long_name, "\\s", "_")
teams$team_long_name = str_replace_all(teams$team_long_name, "\\.", "_")
teams$team_long_name = str_replace_all(teams$team_long_name, "-", "_")

In [8]:
head(teams)

id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
1,9987,673,KRC_Genk,GEN
2,9993,675,Beerschot_AC,BAC
3,10000,15005,SV_Zulte_Waregem,ZUL
4,9994,2007,Sporting_Lokeren,LOK
5,9984,1750,KSV_Cercle_Brugge,CEB
6,8635,229,RSC_Anderlecht,AND


In [9]:
# Agrupando as equipes por paés
CountryClub = Matches %>% 
  group_by(home_team_api_id,country_id) %>% 
  summarise(n=n()) %>% 
  left_join(league) %>%
  left_join(teams, by=c("home_team_api_id" = "team_api_id"))

`summarise()` regrouping output by 'home_team_api_id' (override with `.groups` argument)
Joining, by = "country_id"


# Preparando os dados para mineração das regras de associação

In [10]:
# Os jogadores estão em colunas separadas, mas precisamos deles empilhados em uma coluna
tmp = Matches %>% 
  select(
    season, 
    home_team_api_id, 
    home_player_1:home_player_11
  )%>%
  gather(
    player, 
    player_api_id, 
    -c(season, home_team_api_id)
  ) %>%
  group_by(player_api_id, home_team_api_id ) %>% 
  summarise(season = min(season))

`summarise()` regrouping output by 'player_api_id' (override with `.groups` argument)


In [11]:
# Unindo dados de jogador e clube
playerClubSequence = left_join(
  tmp,
  players
) %>% 
  left_join(
    teams, 
    by=c("home_team_api_id"="team_api_id")
  )

playerClubSequence = playerClubSequence %>% 
  filter(
    !is.na(player_name), !is.na(team_short_name)
  )  %>%
  arrange(
    player_api_id, 
    season
  )

Joining, by = "player_api_id"


In [12]:
# Adicionando um número sequencial por jogador
playerClubSequence$seqnr = ave( playerClubSequence$player_api_id, playerClubSequence$player_api_id, FUN = seq_along)
playerClubSequence$size = 1

# Mineração de sequências com algoritmo cSPade do pacote arulesSequences

In [13]:
# Grava o conjunto de dados em um arquivo txt para facilitar a manipulação 
# da função read_basket em arulesSequence para criar um objeto de transação
write_delim( 
  playerClubSequence %>% select( c(player_api_id, seqnr, size, team_long_name)) ,
  delim ="\t", path = "player_transactions.txt", col_names = FALSE
)

In [14]:
# Agora importamos as transações registradas no item anterior
playerstrxs <- read_baskets("player_transactions.txt", sep = "[ \t]+",info =  c("sequenceID","eventID","size"))
summary(playerstrxs)

transactions as itemMatrix in sparse format with
 17570 rows (elements/itemsets/transactions) and
 296 columns (items) and a density of 0.003378378 

most frequent items:
             Genoa  VitÃ³ria_SetÃºbal         Kilmarnock           SC_Braga 
               131                114                113                108 
Standard_de_LiÃ¨ge            (Other) 
               108              16996 

element (itemset/transaction) length distribution:
sizes
    1 
17570 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      1       1       1       1       1       1 

includes extended item information - examples:
                labels
1          1__FC_KÃ¶ln
2 1__FC_Kaiserslautern
3      1__FC_NÃ¼rnberg

includes extended transaction information - examples:
  sequenceID eventID size
1       2625       1    1
2       2625       2    1
3       2625       3    1

In [15]:
# Executar mineração de sequência, por enquanto apenas com comprimento de duas sequências
playersClubSeq <- cspade(
  playerstrxs, 
  parameter = list(support = 0.00010, maxlen=2), 
  control   = list(verbose = TRUE)
)


parameter specification:
support : 1e-04
maxsize :    10
maxlen  :     2

algorithmic control:
bfstype  : FALSE
verbose  :  TRUE
summary  : FALSE
tidLists : FALSE

preprocessing ... 1 partition(s), 0.41 MB [0.14s]
mining transactions ... 0.04 MB [0.19s]
reading sequences ... [0.14s]

total elapsed time: 0.47s


In [16]:
summary(playersClubSeq)

set of 2478 sequences with

most frequent items:
Fiorentina      Genoa       Roma  Sampdoria      Milan    (Other) 
        52         49         44         43         42       4430 

most frequent elements:
{Fiorentina}      {Genoa}       {Roma}  {Sampdoria}    {Bologna}      (Other) 
          52           49           44           43           42         4430 

element (sequence) size distribution:
sizes
   1    2 
 296 2182 

sequence length distribution:
lengths
   1    2 
 296 2182 

summary of quality measures:
    support         
 Min.   :0.0001897  
 1st Qu.:0.0001897  
 Median :0.0001897  
 Mean   :0.0008981  
 3rd Qu.:0.0003794  
 Max.   :0.0124253  

includes transaction ID lists: FALSE 

mining info:
        data ntransactions nsequences support
 playerstrxs         17570      10543   1e-04

# Fazendo Data Wrangling para colocar os resultados do cspade em um organizado conjunto de dados 

In [17]:
# que é adequado para a visNetwork. A visNetwork precisa de dois conjuntos de dados:
# um conjunto de dados com as arestas "de --> para" e um conjunto de dados com os nós exclusivos
seqResult = as(playersClubSeq, "data.frame")
seqResult = seqResult %>% 
  mutate(
    sequence = as.character(sequence)
  )

seqResult = bind_cols(
  seqResult,
  as.data.frame(
    str_split_fixed(seqResult$sequence, pattern =",", 2), 
    stringsAsFactors = FALSE)
)

seqResult$from = str_extract_all(seqResult$V1,"\\w+", simplify = TRUE)[,1] 
seqResult$to   = str_extract_all(seqResult$V2,"\\w+",simplify = TRUE)[,1]


seqResult$width = exp(3000*seqResult$support)
seqResult = seqResult %>% filter(V2 !="")
seqResult$title = paste(seqResult$sequence, "<br>", round(100*seqResult$support,2), "%")

seqResult$support_perc = paste(sprintf("%.4f", 100*seqResult$support), "%")

In [18]:
# Criando o dataframe com os nodes
nodes = unique(c(seqResult$from, seqResult$to))
nodesData = data.frame(id = unique(nodes), title = unique(nodes), label = unique(nodes), stringsAsFactors = FALSE) %>%
  left_join(CountryClub, by = c("id"="team_long_name")) %>% 
  rename(group = name)

In [19]:
head(nodes)

[1] "UniÃ"             "ÅšlÄ"             "Jagiellonia_BiaÅ" "Ruch_ChorzÃ"     
[5] "Widzew_Å"         "GKS_BeÅ"

In [20]:
# Calcula as medidas de centralidade de betweeness
# usando o igraph, para que possamos ter tamanhos diferentes de
# nós no gráfico de rede
transferGraph = graph_from_data_frame(seqResult[,c(5,6)], directed = TRUE)

tmp = betweenness(transferGraph)
Clubs_betweenness = data.frame(id = names(tmp), value = tmp, stringsAsFactors = FALSE)
nodesData = nodesData %>% 
  left_join(Clubs_betweenness) %>%
  mutate(title = paste(id, "betweeness ", round(value))) %>%
  arrange(id)

Joining, by = "id"


# Criando a rede interativa

In [21]:
# Preparando o dataframe final e removendo duplicidades
nodes = nodesData
nodes = nodes[!duplicated(nodes$id),]

In [22]:
# Cria a rede
visNetwork(nodes, edges = seqResult, width = 900, height = 700) %>%
  visNodes(size = 10) %>%
  visLegend() %>%
  visEdges(smooth = FALSE) %>%
  visOptions(highlightNearest = TRUE, nodesIdSelection = TRUE) %>%
  visInteraction(navigationButtons = TRUE) %>%
  visEdges(arrows = 'from') %>%
  visPhysics(
    solver = "barnesHut",
    maxVelocity = 35,
    forceAtlas2Based = list(gravitationalConstant = -6000)
  )

HTML widgets cannot be represented in plain text (need html)

In [23]:
# Cria a tabela final para suportar a análise
seqResult$Ntransctions = seqResult$support*10542
DT::datatable(
  seqResult[,c(5,6,9,10)], 
  rownames = FALSE,
  options = list(
    pageLength=25)
)

HTML widgets cannot be represented in plain text (need html)